# Decryption with Heuristics and Machine Learning

In [5]:
import pandas as pd
import re
from collections import Counter

# read processed data
cyphertext_path = '../data/processed/countdown_23.csv'
cyphertext_df = pd.read_csv(cyphertext_path)

# read reference artists
with open('../data/artists.txt', 'r') as file:
    reference_artists = file.read().splitlines()

In [6]:
# heuritic patterns
dj_set_pattern = re.compile(r"\([A-Z0-9]{2} [A-Z0-9]{3}\)")
b2b_pattern = re.compile(r"\b(\w+\s)([A-Z0-9])([A-Z0-9])(\2)(\s\w+)")


def extract_patterns(text):
    mappings = {}
    # DJ SET
    dj_set_matches = dj_set_pattern.findall(text)
    for match in dj_set_matches:
        cypher_dj_set = match[1:-1]
        plain_dj_set = "DJ SET"
        for c, p in zip(
            cypher_dj_set.replace(" ", ""), plain_dj_set.replace(" ", "")
        ):
            mappings[c] = p

    # B2B
    b2b_matches = b2b_pattern.findall(text)
    for match in b2b_matches:
        cypher_b2b = match[1] + match[2]
        plain_b2b = "B2"
        for c, p in zip(cypher_b2b, plain_b2b):
            mappings[c] = p

    return mappings